# Photometry Analysis

The goal of this script is to:

1. import photometry data and crop
1. deinterleave data
1. normalize photometry data for photobleaching
1. chop up photometry data around keystrokes


## import and definition


In [ ]:
from IPython.display import display
from ipywidgets import widgets

from routine.plotting import plot_events, plot_signals
from routine.processing import photobleach_correction
from routine.utilities import load_data, pool_events

IN_DATA = "./data/data.csv"
IN_TS = "./data/timestamp.csv"
PARAM_ROIS = ["Region0G"]
PARAM_NFM_DISCARD = 100
PARAM_LED_DICT = {7: "initial", 1: "415nm", 2: "470nm", 4: "560nm"}
PARAM_EVT_RANGE = (-200, 400)
PARAM_BASE_SIG = "415nm"

## load data


ensure equal number of frames for each channel


In [ ]:
data, ts = load_data(IN_DATA, IN_TS, PARAM_NFM_DISCARD, PARAM_LED_DICT)


## visualize raw signal


In [ ]:
plot_signals(data, PARAM_ROIS)


## correcting for photobleaching


order of operations:

1. deinterleave data by flag (LED) and save into a matrix
1. fit isosbestic signal with a biexponential decay -- the shape of this decay is a good approximation of the CONCENTRATION of GCaMP molecules underneath your fiber.
   it decreases as the photobleach.
   the amplitude, however, is tiny.
   to adjust for this, we:
1. linearly scale the fitted decay to the 470 data using robust fit.
1. divide the raw 470 data by this scale fit to get a corrected signal

note: this isn't dF/F but it is INTERNALLY reliable -- that is, you can compare the beginning of the recording to the end of the recording.
dF/F requires a good approximation of baseline.
you can use the `FP.no_led` as an underestimation of this -- or determine it empirically.
it often isn't critical to a sound analysis.


In [ ]:
data_norm = photobleach_correction(data, PARAM_ROIS, PARAM_BASE_SIG)


## visualize correction result


In [ ]:
plot_signals(data_norm, PARAM_ROIS)


## pool signals around `'Key'` events


In [ ]:
evt_df = pool_events(ts, data, PARAM_EVT_RANGE, PARAM_ROIS, event_name="Key")


## visualize signals around events


In [ ]:
plot_events(evt_df)
